In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

In [ ]:
data2001 = pd.read_csv('./pems04/d04_text_station_5min_2018_02_28.txt.gz',header=None, usecols=range(12))
# Assign column names based on the provided headers
column_names = [
    "Timestamp", "Station", "District", "Freeway #", 
    "Direction of Travel", "Lane Type", "Station Length", 
    "Samples", "% Observed", "Total Flow", "Avg Occupancy", "Avg Speed"
]

# Assign column names to the dataframe
data2001.columns = column_names
data2001

In [ ]:
comm = np.load('pems04_comm.npy')
comm

In [ ]:
import pandas as pd
import calendar
from datetime import date, timedelta

def read_and_process_data(file_path):
    # Read the CSV file, specifying the usecols parameter to only load the columns of interest
    data = pd.read_csv(
        file_path, 
        header=None, 
        usecols=[0, 1, 9],  # Column indexes for Timestamp, Station, and Total Flow
        names=['Timestamp', 'Station', 'Avg Flow'],  # Assigning column names
        compression='gzip'
    )

    # Convert the Timestamp column to datetime format and set it as the index
    data['Timestamp'] = pd.to_datetime(data['Timestamp'])
    data.set_index('Timestamp', inplace=True)

    # Pivot the table to get Stations as columns and Total Flow as cell values
    pivot_data = data.pivot(columns='Station', values='Avg Flow')

    return pivot_data

# Read the data for February 28 to get the column names (Station IDs)
feb_28_data = read_and_process_data('./pems04/d04_text_station_5min_2018_02_28.txt.gz')
feb_28_columns = feb_28_data.columns

# Initialize a list to hold the data for all days
all_data = []

start_date = date(2002, 6, 1)  # 开始日期
end_date = date(2024, 3, 20)   # 结束日期
current_date = start_date
all_data = []                  # 存储所有数据的列表

while current_date <= end_date:
    year = current_date.year
    month = current_date.month
    day = current_date.day
    file_path = f'./pems04/d04_text_station_5min_{year}_{month:02d}_{day:02d}.txt.gz'
    
    # 检查文件是否存在
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}, skipping...")
        current_date += timedelta(days=1)
        continue
    
    day_data = read_and_process_data(file_path)
    
    # 使用字典存储新列，以便一次性添加到数据帧
    new_columns = {}
    for col in comm:
        if col not in day_data:
            new_columns[col] = 0  # 假设使用0填充缺失的列

    # 使用 pd.concat 添加所有新列
    if new_columns:
        new_data = pd.DataFrame(new_columns, index=day_data.index)
        day_data = pd.concat([day_data, new_data], axis=1)

    # 将列重新排序以匹配2月28日的顺序
    day_data = day_data[comm]
    
    all_data.append(day_data)
    
    current_date += timedelta(days=1)

# Concatenate all daily data into a single DataFrame
combined_data = pd.concat(all_data)

# Fill any remaining missing values with 0
combined_data.fillna(0, inplace=True)

# The combined_data DataFrame now holds the merged data for January 1 to February 28
# with columns aligned to February 28 and missing values filled with 0
# Note: The code execution is commented out to prevent execution in this environment.
# combined_data.head()
combined_data.to_csv('pems04_all_common_flow.csv')

In [ ]:
combined_data

In [2]:
import pandas as pd
pems04 = pd.read_csv('pems04_all_common_flow.csv')

# Convert the 'date' column to datetime
pems04['date'] = pd.to_datetime(pems04['date'])

# Set the 'date' column as the index
pems04.set_index('date', inplace=True)

# Resample to hourly data and aggregate using sum
pems04_hourly = pems04.resample('H').sum()

# Reset the index if you want the 'date' column back
pems04_hourly.reset_index(inplace=True)
pems04_hourly.to_csv('pems04_h.csv',index=False)


# Convert the 'date' column to datetime
pems04 = pd.read_csv('pems04_all_common_flow.csv')
pems04['date'] = pd.to_datetime(pems04['date'])

# Set the 'date' column as the index
pems04.set_index('date', inplace=True)

# Resample to hourly data and aggregate using sum
pems04_daily = pems04.resample('D').sum()

# Reset the index if you want the 'date' column back
pems04_daily.reset_index(inplace=True)
pems04_daily.to_csv('pems04_d.csv',index=False)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_51680\3748972102.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  pems04_hourly = pems04.resample('H').sum()
